# Energy System Modelling - Tutorial I.1

Karlsruhe Institute of Technology, Institute for Automation and Applied Informatics
***


# Imports

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('bmh')
%matplotlib inline

***
# Introductory Comments

## Getting Help

Executing cells with Shift-Enter and with `h` there is help.

Help is available with `.<TAB>` or `load.sort_values()` <- cursor between brackets, `Shift-<TAB>`

## Using one-dimensional arrays (Numpy and Pandas)

**Numpy**

In [4]:
a = np.arange(10)
a

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [5]:
a[1:3]

array([1, 2])

**Pandas**

In [6]:
s = pd.Series(np.random.random(3), index=['foo', 'bar', 'baz'])
s

foo    0.162113
bar    0.772345
baz    0.743816
dtype: float64

In [7]:
s["foo":"bar"]

foo    0.162113
bar    0.772345
dtype: float64

## Using two-dimensional arrays (Numpy and Pandas)

**Numpy** 

In [8]:
np.random.random((3,5))

array([[0.8761608 , 0.95568137, 0.11391783, 0.55694616, 0.08709893],
       [0.42898427, 0.646773  , 0.34092153, 0.82829308, 0.61801058],
       [0.88191614, 0.49017959, 0.73795784, 0.19966381, 0.3970734 ]])

**Pandas**

In [9]:
s = pd.DataFrame(np.random.random((3,5)),
                 index=['foo', 'bar', 'baz'],
                 columns=['colA', 'colB', 'colC', 'colD', 'colE'])
s

,colA,colB,colC,colD,colE
foo,0.344249,0.680323,0.827116,0.578043,0.132042
bar,0.086543,0.402303,0.625985,0.478828,0.698698
baz,0.410675,0.181072,0.309318,0.888258,0.544812


In [10]:
s.mean()

colA    0.280489
colB    0.421233
colC    0.587473
colD    0.648376
colE    0.458517
dtype: float64

In [11]:
s.mean(axis=1)

foo    0.512355
bar    0.458471
baz    0.466827
dtype: float64

***
# Problem I.1

The following data are made available to you on the repository in the `./data` directory:

`de_data.csv`, `gb_data.csv`, `eu_data.csv`
and alternatively
`wind.csv`, `solar.csv`, `load.csv`

They describe (quasi-real) time series for wind power generation $W(t)$, solar power generation $S(t)$ and load $L(t)$ in Great Britain (GB), Germany (DE) and Europe (EU). The time step is 1 h and the time series are several years long.

> Remark: In this example notebook, we only look at Germany and the EU, Great Britain works in exactly the same way.

***
**Read Data**

In [12]:
de = pd.read_csv('data/de_data.csv', parse_dates=True, index_col=0)
eu = pd.read_csv('data/eu_data.csv', parse_dates=True, index_col=0)
gb = pd.read_csv('data/gb_data.csv', parse_dates=True, index_col=0)

In [13]:
wind = pd.read_csv('data/wind.csv', parse_dates=True, index_col=0)
solar = pd.read_csv('data/solar.csv', parse_dates=True, index_col=0)
load = pd.read_csv('data/load.csv', parse_dates=True, index_col=0)

Extra: Show the first and last 5 lines (header) of the German data:

In [14]:
de.head()

,wind,solar,load
time,,,
2011-01-01 00:00:00,0.535144,0.0,46209.0
2011-01-01 01:00:00,0.580456,0.0,44236.0
2011-01-01 02:00:00,0.603605,0.0,42502.0
2011-01-01 03:00:00,0.614114,0.0,41479.0
2011-01-01 04:00:00,0.627257,0.0,39923.0


In [15]:
de.tail()

,wind,solar,load
time,,,
2014-12-31 19:00:00,0.191347,0.0,50365.0
2014-12-31 20:00:00,0.220209,0.0,48725.0
2014-12-31 21:00:00,0.247598,0.0,49074.0
2014-12-31 22:00:00,0.273812,0.0,47667.0
2014-12-31 23:00:00,0.295076,0.0,47667.0


Extra: Check that wind, solar and load files are just differently organized datasets and it's the same data:

In [16]:
(wind['DE'] == de['wind']).all()

True

Extra: How many years does the dataset comprise?

In [17]:
de.index

DatetimeIndex(['2011-01-01 00:00:00', '2011-01-01 01:00:00',
               '2011-01-01 02:00:00', '2011-01-01 03:00:00',
               '2011-01-01 04:00:00', '2011-01-01 05:00:00',
               '2011-01-01 06:00:00', '2011-01-01 07:00:00',
               '2011-01-01 08:00:00', '2011-01-01 09:00:00',
               ...
               '2014-12-31 14:00:00', '2014-12-31 15:00:00',
               '2014-12-31 16:00:00', '2014-12-31 17:00:00',
               '2014-12-31 18:00:00', '2014-12-31 19:00:00',
               '2014-12-31 20:00:00', '2014-12-31 21:00:00',
               '2014-12-31 22:00:00', '2014-12-31 23:00:00'],
              dtype='datetime64[ns]', name='time', length=35064, freq=None)

Data set includes four years ranging from `2011-01-01` until `2014-12-31`.

***
**(a) Check that the wind and solar time series are normalized to ’per-unit of installed capacity’,
and that the load time series is normalized to MW.**

In [20]:
eu.max()

wind          0.719222
solar         0.808031
load     539904.000000
dtype: float64

***
**(b) For all three regions, calculate the maximum, mean, and variance of the time series.**

In [22]:
pd.concat([eu, de, gb], axis=1).describe()

,wind,solar,load,wind,solar,load,wind,solar,load
count,35064.000000,35064.000000,35064.000000,35064.000000,35064.000000,35064.000000,35064.000000,35064.000000,35064.000000
mean,0.238825,0.156654,357003.275140,0.265785,0.149262,54877.199806,0.377176,0.124691,34573.316678
std,0.124324,0.210926,61235.523002,0.239194,0.221564,10354.693128,0.280585,0.195193,7475.509068
min,0.022194,0.000000,222327.000000,0.000577,0.000000,29201.000000,0.002075,0.000000,18159.000000
25%,0.141962,0.000000,309072.000000,0.078827,0.000000,46321.750000,0.137268,0.000000,28860.000000
50%,0.217789,0.012116,358094.500000,0.181354,0.000000,54594.000000,0.301760,0.000000,34843.000000
75%,0.316816,0.308416,397490.250000,0.394514,0.268505,63953.250000,0.586097,0.202772,39577.000000
max,0.719222,0.808031,539904.000000,0.994588,0.913781,79286.000000,0.999998,0.888284,55614.000000


***
**(c) For all three regions, plot the time series $W (t)$, $S(t)$, $L(t)$ for a winter month (January) and a summer month (July).**

Extra: Also compare the wind between the different regions

***
**(d) Resample the time series to daily, weekly and monthly data points and visualise them in plots. Can you identify some recurring patterns?**

> **Hint:** Use the function [`.resample`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.resample.html) with `.mean`.

Wind:

Solar:

Load:

***
**(e) For all three regions, plot the duration curve for $W(t)$, $S(t)$, $L(t)$.** 
> **Hint:** You might want to make use of the functions [`.sort_values`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sort_values.html) and [`.reset_index`](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.DataFrame.reset_index.html)

> **Tip:** Go through the line `de['wind'].sort_values(ascending=False).reset_index(drop=True).plot()` dot by dot and note what happens to the output.

***
**(f) For all three regions, plot the probability density function of $W(t)$, $S(t)$, $L(t)$.**

There are two different methods:
1. [Histograms](https://en.wikipedia.org/wiki/Histogram) and 
2. [Kernel density estimation (KDE)](https://en.wikipedia.org/wiki/Kernel_density_estimation).

This [image](https://en.wikipedia.org/wiki/Kernel_density_estimation#/media/File:Comparison_of_1D_histogram_and_KDE.png) on the KDE page provides a good summary of the differences. You can do both with `pandas`!

First, let's look at the wind data:

Now, let's look at the solar data:

The solar data might be hard to see. Look at this in detail by limiting the y-axis to (0,2):

Finally, let's look at the load profile:

***
**(g) Apply a [(Fast) Fourier Transform](https://en.wikipedia.org/wiki/Fast_Fourier_transform) to the the three time series $X \in W(t), S(t), L(t)$:**

$$\tilde{X}(\omega) = \int_0^T X(t) \;e^{i\omega t} \;\mathrm{d}t.$$

**For all three regions, plot the energy spectrum $\|\tilde{X}(\omega)\|^2$ as a function of $\omega$. Discuss the relationship of these results with the findings obtained in (b)-(f).**

> **Remark:** Use the function [`numpy.fft.rfft`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.fft.rfft.html) and make sure you subtract the mean since the Fourier transformation requires the time series to have a zero mean to decompose it into its frequencies.

> **Remark:** To determine the frequencies [`numpy.fft.rfffreq`](https://docs.scipy.org/doc/numpy-1.12.0/reference/generated/numpy.fft.rfftfreq.html) is used, the argument `d` indicates the distance between two data points, `1h` hour, which we specify as $\frac{1}{8760} a$, so that the frequencies come out in the unit $\frac{1}{a}$.

***
**(h) Normalize the time series to one, so that $\langle{W}\rangle = \langle{S}\rangle = \langle{L}\rangle = 1$.**

**Now, for all three regions, plot the mismatch time series**
  
  $$\Delta(t) = \gamma \alpha W(t) + \gamma (1 - \alpha) S(t) - L(t) $$
  
**for the same winter and summer months as in (c). Choose** 
1. $\alpha \in \{0.0, 0.5, 0.75, 1.0\}$ with $\gamma = 1$, and 
2. $\gamma \in \{0.5, 0.75, 1.0, 1.25, 1.5\}$ with $\alpha = 0.75$.

**What is the interpretation of $\gamma$ and $\alpha$?**

**Which configuration entails the lowest mismatch on average and in extremes?**

Choose the country and alpha, gamma values and re-run:

In [25]:
d = de
gamma = 1.0
alpha = 0

Normalize the time series and calculate mismatch time series:

Plot the mismatch time series for the winter and summer months:

***
**(i) For all three regions, repeat (b)-(g) for the mismatch time series. What changed?**

**Statistics**

**Time series plot**

**Duration curve**

**Probability density function**

**Fast Fourier Transform**